# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [50]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
import scipy.stats as st
import gmaps
import os

# Import API key
from api_keys import g_key
from api_keys import weather_api_key
gmaps.configure(api_key=g_key)

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [51]:
cities_csv = "output_data/cities.csv"
cities_df = pd.read_csv(cities_csv)

cities_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Qaanaaq,77.4840,-69.3632,38.07,76,59,2.01,GL,1624700297
1,1,Miles City,46.4083,-105.8406,58.17,62,75,3.44,US,1624700297
2,2,San Policarpo,12.1791,125.5072,83.39,72,100,2.21,PH,1624700298
3,3,Sitka,57.0531,-135.3300,55.29,88,90,5.75,US,1624700298
4,4,Lebu,-37.6167,-73.6500,43.77,81,6,7.63,CL,1624700299
...,...,...,...,...,...,...,...,...,...,...
586,586,Bonnyville,54.2668,-110.7350,58.80,79,74,2.82,CA,1624700517
587,587,Manali,13.1667,80.2667,93.29,61,79,5.75,IN,1624700517
588,588,Kristinestad,62.2743,21.3760,67.64,57,72,13.02,FI,1624700518
589,589,Khanbari,27.3747,87.2039,82.94,74,75,5.06,NP,1624700518


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [52]:
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]

heatmaps_fig = gmaps.figure(zoom_level=2, center=(30, 0))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

heatmaps_fig.add_layer(heat_layer)

heatmaps_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [53]:
ideal_cities = cities_df.loc[cities_df["Max Temp"] < 80]
ideal_cities = ideal_cities[ideal_cities["Max Temp"] > 70]
ideal_cities = ideal_cities[ideal_cities["Cloudiness"] == 0]

ideal_cities

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,Aljezur,37.3191,-8.8033,77.47,47,0,5.77,PT,1624700303
17,17,Rundu,-17.9333,19.7667,78.31,18,0,5.46,NaN,1624700303
31,31,Dekar,-21.5333,21.9333,74.95,20,0,5.84,BW,1624700309
34,34,Tiznit Province,29.5833,-9.5000,78.42,33,0,5.50,MA,1624700310
80,80,Linguère,15.3953,-15.1193,79.02,83,0,4.61,SN,1624700326
125,125,Marrakesh,31.6315,-8.0083,77.07,57,0,2.30,MA,1624700104
272,272,Redmond,47.6740,-122.1215,73.87,77,0,1.99,US,1624700394
296,296,Saurimo,-9.6608,20.3916,79.12,41,0,8.05,AO,1624700408
359,359,Arroyo de la Encomienda,41.6096,-4.7969,78.73,40,0,11.50,ES,1624700432
441,441,Brownwood,31.7093,-98.9912,78.57,84,0,18.63,US,1624700243


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
hotel_df = ideal_cities

hotel_df["Hotel Name"] = ""

hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
16,16,Aljezur,37.3191,-8.8033,77.47,47,0,5.77,PT,1624700303,
17,17,Rundu,-17.9333,19.7667,78.31,18,0,5.46,NaN,1624700303,
31,31,Dekar,-21.5333,21.9333,74.95,20,0,5.84,BW,1624700309,
34,34,Tiznit Province,29.5833,-9.5000,78.42,33,0,5.50,MA,1624700310,
80,80,Linguère,15.3953,-15.1193,79.02,83,0,4.61,SN,1624700326,
125,125,Marrakesh,31.6315,-8.0083,77.07,57,0,2.30,MA,1624700104,
272,272,Redmond,47.6740,-122.1215,73.87,77,0,1.99,US,1624700394,
296,296,Saurimo,-9.6608,20.3916,79.12,41,0,8.05,AO,1624700408,
359,359,Arroyo de la Encomienda,41.6096,-4.7969,78.73,40,0,11.50,ES,1624700432,
441,441,Brownwood,31.7093,-98.9912,78.57,84,0,18.63,US,1624700243,


In [55]:
target_search = "hotel"
radius = 5000
names = []

for index, row in hotel_df.iterrows():
    lat = row.loc["Lat"]
    lng = row.loc["Lng"]
    coords = f"{lat}, {lng}"

    params = {
        "location":  coords,
        "keyword": target_search,
        "radius": radius,
        "key": g_key
        }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_name = requests.get(base_url, params=params).json()
    #print(json.dumps(hotel_name, indent=4, sort_keys=True))

    #print(hotel_name)
    #print("")

    counter = 0

    if len(hotel_name["results"]) != 0:
        for place in hotel_name["results"]:
            #print(place["name"])
            names.append(place["name"])
            #print("")
            counter += 1

            if counter == 1:
                break

    else:
        #print("N/A")
        names.append("")
        #print("")

hotel_df["Hotel Name"] = names
hotel_df


    

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
16,16,Aljezur,37.3191,-8.8033,77.47,47,0,5.77,PT,1624700303,Herdade Monte Do Sol
17,17,Rundu,-17.9333,19.7667,78.31,18,0,5.46,NaN,1624700303,Tambuti Lodge
31,31,Dekar,-21.5333,21.9333,74.95,20,0,5.84,BW,1624700309,
34,34,Tiznit Province,29.5833,-9.5000,78.42,33,0,5.50,MA,1624700310,
80,80,Linguère,15.3953,-15.1193,79.02,83,0,4.61,SN,1624700326,AYO Casablanca
125,125,Marrakesh,31.6315,-8.0083,77.07,57,0,2.30,MA,1624700104,Sofitel Marrakech Lounge & Spa
272,272,Redmond,47.6740,-122.1215,73.87,77,0,1.99,US,1624700394,Archer Hotel Redmond
296,296,Saurimo,-9.6608,20.3916,79.12,41,0,8.05,AO,1624700408,Resort Uhenha
359,359,Arroyo de la Encomienda,41.6096,-4.7969,78.73,40,0,11.50,ES,1624700432,AC Hotel by Marriott Palacio de Santa Ana
441,441,Brownwood,31.7093,-98.9912,78.57,84,0,18.63,US,1624700243,"Holiday Inn Express Early, an IHG Hotel"


In [56]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [57]:
# Add marker layer ontop of heat map

hotel_info = [info_box_template.format(**row) for index,row in hotel_df.iterrows()]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
heatmaps_fig.add_layer(marker_layer)
heatmaps_fig


# Display figure


Figure(layout=FigureLayout(height='420px'))